In [213]:
import requests as req
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wb
import time
import numpy as np
import pandas as pd
import os # 폴더 제어
from selenium.webdriver.common.keys import Keys # 키보드 제어
from tqdm import tqdm # 진행상황
from urllib.request import urlretrieve # 이미지 다운로드 도구
from selenium.webdriver.chrome.options import Options
import json
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from collections import Counter
from konlpy.tag import Okt

In [276]:
data = pd.read_csv('./JeJu.csv', encoding = "euc-kr")
data

이름     태그1     태그2    태그3  \
0                 우도   섬속의섬   경관/포토      아이   
1              성산일출봉     일출      오름   경관/포토   
2              사려니숲길     숲길   걷기/등산      친구   
3              카멜리아힐  경관/포토      커플      아이   
4             협재해수욕장     일몰    해수욕장    액티비티   
...              ...     ...     ...    ...   
1078  황우럭만화천국사회적협동조합     친구      아이    체험관광   
1079             효명사  경관/포토   휴식/힐링       절   
1080            후포해변     해변    경관/포토    NaN   
1081     휘닉스 르쏠레이테라피    웰니스      겨울      스파   
1082              휴림     부모      친구      아이   

                                                  주소  \
0                           제주특별자치도 제주시 우도면 삼양고수물길 1   
1                        제주특별자치도 서귀포시 성산읍 일출로 284-12   
2                        제주특별자치도 제주시 조천읍 교래리 산 137-1   
3                                제주 서귀포시 안덕면 병악로 166   
4                         제주특별자치도 제주시 한림읍 한림로 329-10   
...                                              ...   
1078                        제주특별자치도 제주시 한림읍 내동남길 7-3   
1079                    제주특별자치도 서귀포시 남원읍 516로 815-41   
1080                         제주특별자치도 제주시 추자면 대서5길 87   
1081  제주특별자치도 서귀포시 성산읍 섭지코지로 107 휘닉스제주아일랜드리조트 블루동 1층   
1082                        제주특별자치도 제주시 애월읍 광령남서길 40   

                                                    이미지  \
0     //api.cdn.visitjeju.net/photomng/imgpath/20190...   
1     //api.cdn.visitjeju.net/photomng/imgpath/20181...   
2     //api.cdn.visitjeju.net/photomng/imgpath/20190...   
3     //api.cdn.visitjeju.net/photomng/imgpath/20200...   
4     //api.cdn.visitjeju.net/photomng/imgpath/20191...   
...                                                 ...   
1078  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
1079  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
1080  //api.cdn.visitjeju.net/photomng/imgpath/20180...   
1081  //api.cdn.visitjeju.net/photomng/imgpath/20201...   
1082  //api.cdn.visitjeju.net/photomng/imgpath/20190...   

                                                     설명        lat        long  
0                        소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬  33.519491  126.951090  
1      바다위에 우뚝 솟아난 수성화산·유네스코 세계자연유산, 천연기념물 제420호, 올레1코스  33.462147  126.936427  
2               제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길  33.423005  126.626868  
3     카멜리아힐은 동양에서 가장 큰 동백 수목원으로 토종 동백부터 아기 동백, 유럽 동백...  33.290114  126.368404  
4                        비양도, 은모래, 바다가 그려낸 그림같이 아름다운 해변  33.393846  126.239714  
...                                                 ...        ...         ...  
1078         황우럭 만화카페는 故양병윤 화백의 생가를 카페로 새롭게 재구성한 장소입니다.  33.414267  126.266673  
1079                                 한라산 중턱에 위치한 소박한 사찰  33.322902  126.594584  
1080                               탈의실과 편의시설을 갖춘 작은 휴양지  33.962519  126.292059  
1081  스파, 커플룸, 프라이빗룸으로 구성된 르쏠레이테라피센터는 성산 휘닉스호텔앤리조트 내...  33.430540  126.927739  
1082                        제주시 산림조합이 조성한 숲속 쉼터, 에코힐링파크  33.443436  126.428992  

[1083 rows x 9 columns]

In [238]:
# 네이버 검색 API예제는 블로그를 비롯 전문자료까지 호출방법이 동일하므로 blog검색만 대표로 예제를 올렸습니다.
# 네이버 검색 Open API 예제 - 블로그 검색
import os
import sys
import urllib.request
client_id = "gkCvxek6W15lu0nkdYYB"
client_secret = "64E2y8DfEG"
encText = urllib.parse.quote("우도 여행")
url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{
"lastBuildDate": "Thu, 17 Jun 2021 09:52:41 +0900",
"total": 312706,
"start": 1,
"display": 10,
"items": [
{
"title": "[제주<b>여행</b> day5 03.17] 감성 가득한 섬속의 섬 <b>우도</b>",
"link": "https:\/\/blog.naver.com\/kyhfx?Redirect=Log&logNo=222393782123",
"description": "비양도의 멋진 풍경도 감상을 한 뒤 마지막으로 땅콩아이스크림과 망고쥬스를 친 뒤 <b>우도 여행</b>을 마무리합니다. 제주도에서 사먹은 쥬스 천혜향쥬스, 망고쥬스 등등 다 맛있었어요. 이번 제주 여행중에 우도는... ",
"bloggername": "[Make free use a dream] JK PICTURES",
"bloggerlink": "https://blog.naver.com/kyhfx",
"postdate": "20210611"

},
{
"title": "투어로 떠나보는 제주 <b>우도 여행</b>",
"link": "https:\/\/blog.naver.com\/happylife_mi?Redirect=Log&logNo=222295453316",
"description": "생각 덜어내기 여행이었기 때문에 차량 대여 보다는 가벼운 방법을 고심하다가, 여러 곳 중에서도... 봉끄랑 주소 : 제주시 우도면 우도해안길 144 제주 <b>우도 여행</b>은 자유 일정이기 때문에 섬에 들어오자마자... ",
"bloggername": "동팔이의 즐거운상삶",
"bloggerlink": "https://blog.naver.com/happylife_mi",
"postdate": "20210401"

},
{
"title": "제주도 2박3일 렌트카 없이 <b>여행</b> <b>우도</b> 코스",
"link": "https:\/\/blog.naver.com\/wwwlolwww?Redirect=Log

In [239]:
import requests
from urllib.parse import quote
 
def call(keyword, start):
    encText = quote(keyword)
    url = "https://openapi.naver.com/v1/search/blog?query="+encText+"&sort=sim&display=100&start="+str(start)
    result = requests.get(url=url, headers={"X-Naver-Client-Id":"gkCvxek6W15lu0nkdYYB","X-Naver-Client-Secret":"64E2y8DfEG"})
    print(result)  # Response [200]
    return result.json()
 
def results(keyword):
    list = []
    for num in range(0,2):
        list = list + call(keyword, num * 110 + 1)['items'] 
    return list

In [240]:

for name in data['이름']:
    listt = []
    result = results(name+" 여행")
    listt = listt+result
    file = open('coat.json','w+')
    file.write(json.dumps(listt))
    try:
        df = pd.read_json('coat.json')
        df.to_csv('./blog/crawling_naver_{}.csv'.format(name),encoding='utf-8',index=False)
    except:
        pass

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [3]:
def drop_row(row):
    # del row['link']
    del row['description']
    del row['bloggername']
    del row['bloggerlink']
    del row['postdate']

In [4]:
text = pd.DataFrame({
    'title':['title'],
    # 'description':['description'],
    'link':['link']
})
text

title  link
0  title  link

In [5]:
name_list = []
for name in data['이름']:
    try:
        tx = pd.read_csv('./blog/crawling_naver_{}.csv'.format(name))
        drop_row(tx)
        tx['title']=name
        text = pd.concat([text, tx])
    except:
        name_list.append(name)
# text.reset_index(drop=True, inplace = True)
# text.drop(0, inplace=True)
text

title                                               link
0    title                                               link
0       우도  https://blog.naver.com/kyhfx?Redirect=Log&logN...
1       우도  https://blog.naver.com/happylife_mi?Redirect=L...
2       우도  https://blog.naver.com/wwwlolwww?Redirect=Log&...
3       우도  https://blog.naver.com/ranboy?Redirect=Log&log...
..     ...                                                ...
195     휴림  https://blog.naver.com/neburi72?Redirect=Log&l...
196     휴림  https://blog.naver.com/story106?Redirect=Log&l...
197     휴림  https://blog.naver.com/iris3851?Redirect=Log&l...
198     휴림  https://blog.naver.com/dmsrz?Redirect=Log&logN...
199     휴림  https://blog.naver.com/yeppyjoa?Redirect=Log&l...

[198349 rows x 2 columns]

In [6]:
text.drop(range(21,200), inplace=True)
text

title                                               link
0   title                                               link
0      우도  https://blog.naver.com/kyhfx?Redirect=Log&logN...
1      우도  https://blog.naver.com/happylife_mi?Redirect=L...
2      우도  https://blog.naver.com/wwwlolwww?Redirect=Log&...
3      우도  https://blog.naver.com/ranboy?Redirect=Log&log...
..    ...                                                ...
16     휴림  https://blog.naver.com/gayadong?Redirect=Log&l...
17     휴림  https://blog.naver.com/lovemth2?Redirect=Log&l...
18     휴림  https://blog.naver.com/jihee9716?Redirect=Log&...
19     휴림  https://blog.naver.com/hahi11?Redirect=Log&log...
20     휴림  https://blog.naver.com/richguy_gil?Redirect=Lo...

[22561 rows x 2 columns]

In [8]:
text.reset_index(drop=True, inplace = True)
text.drop(0, inplace=True)
text.reset_index(drop=True, inplace = True)
text

title                                               link
0        우도  https://blog.naver.com/kyhfx?Redirect=Log&logN...
1        우도  https://blog.naver.com/happylife_mi?Redirect=L...
2        우도  https://blog.naver.com/wwwlolwww?Redirect=Log&...
3        우도  https://blog.naver.com/ranboy?Redirect=Log&log...
4        우도  https://blog.naver.com/chocobinu215?Redirect=L...
...     ...                                                ...
22555    휴림  https://blog.naver.com/gayadong?Redirect=Log&l...
22556    휴림  https://blog.naver.com/lovemth2?Redirect=Log&l...
22557    휴림  https://blog.naver.com/jihee9716?Redirect=Log&...
22558    휴림  https://blog.naver.com/hahi11?Redirect=Log&log...
22559    휴림  https://blog.naver.com/richguy_gil?Redirect=Lo...

[22560 rows x 2 columns]

In [43]:
# text.reset_index(drop=True, inplace = True)
text_main = text.copy()
text_main

title                                               link
0        우도  https://blog.naver.com/kyhfx?Redirect=Log&logN...
1        우도  https://blog.naver.com/happylife_mi?Redirect=L...
2        우도  https://blog.naver.com/wwwlolwww?Redirect=Log&...
3        우도  https://blog.naver.com/ranboy?Redirect=Log&log...
4        우도  https://blog.naver.com/chocobinu215?Redirect=L...
...     ...                                                ...
22555    휴림  https://blog.naver.com/gayadong?Redirect=Log&l...
22556    휴림  https://blog.naver.com/lovemth2?Redirect=Log&l...
22557    휴림  https://blog.naver.com/jihee9716?Redirect=Log&...
22558    휴림  https://blog.naver.com/hahi11?Redirect=Log&log...
22559    휴림  https://blog.naver.com/richguy_gil?Redirect=Lo...

[22560 rows x 2 columns]

In [41]:
import numpy as np
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as bs
from urllib.request import urlretrieve
from tqdm import tqdm # 진행상황

In [44]:
url = text_main['link'][0]
url

'https://blog.naver.com/kyhfx?Redirect=Log&logNo=222393782123'

In [62]:
text['link'][11183]

'https://blog.naver.com/hanlacho3?Redirect=Log&logNo=221477526701'

In [63]:
text_main['link'][11185:]

11185                                                   []
11186                                                   []
11187    https://photop1.khan.kr/entry/%EC%86%90-%EB%A7...
11188    https://blog.naver.com/howkang?Redirect=Log&lo...
11189    https://blog.naver.com/wingbird2010?Redirect=L...
                               ...                        
22555    https://blog.naver.com/gayadong?Redirect=Log&l...
22556    https://blog.naver.com/lovemth2?Redirect=Log&l...
22557    https://blog.naver.com/jihee9716?Redirect=Log&...
22558    https://blog.naver.com/hahi11?Redirect=Log&log...
22559    https://blog.naver.com/richguy_gil?Redirect=Lo...
Name: link, Length: 11375, dtype: object

In [67]:
header = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
cnt = 0
# text_main = text.copy()
for i in tqdm(text_main['link']):
    url = i
    res = req.get(url, headers = header)
    html = bs(res.text, 'html.parser')
    try:
        target_URL = html.select_one('#mainFrame').get('src')
        target_URL = 'http://blog.naver.com' + target_URL
        res = req.get(target_URL, headers = header)
        html = bs(res.text, 'html.parser')
        blog_text = html.select('div.se-viewer span')
        if blog_text == []:
            blog_text = html.select('')
        blog_list = []
        for b in blog_text:
            list1 = b.text.strip()
            blog_list.append(list1)
        text_main['link'][cnt] = blog_list
    except:
        pass
    cnt +=1

100%|███████████████████████████████████████████████████████████████████████████▉| 11187/11188 [59:50<00:00,  3.12it/s]


SSLError: HTTPSConnectionPool(host='photop1.khan.kr', port=443): Max retries exceeded with url: /entry/%EC%86%90-%EB%A7%9E%EC%9E%A1%EC%9D%80-%EB%82%A8%EB%B6%81%EC%A0%95%EC%83%81-%EB%93%B1-2018-%ED%95%9C%EA%B5%AD%EB%B3%B4%EB%8F%84%EC%82%AC%EC%A7%84-%EC%88%98%EC%83%81%EC%9E%91%EB%93%A4 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1123)')))

In [185]:
text_main['link'][0]

['[제주여행 day5 03.17] 감성 가득한 섬속의 섬 우도',
 'JK',
 '',
 'JK',
 '2021. 6. 11. 11:20',
 '본문 기타 기능',
 '',
 '이번 제주여행의 핵심이라면 핵심이라고 할 수 있는 우도여행입니다.',
 '섬속의 섬',
 '사진과 영상으로만 봤던 에메랄드 빛 바다를 보러 갑니다.',
 '#성산포종합여객터미널 에서 #우도천진항으로 이동을 합니다.',
 '#성산포종합여객터미널',
 '#우도천진항으로',
 '우도가 보이네요.',
 '날씨가 정말 좋았습니다.',
 '구름이 드리운 날씨지만 우도 여행중에는 햇살이 따스하게 내렸습니다.',
 '거리는 멀지 않아 20분? 정도면 도착을 하는듯 합니다.',
 '설레옵니다~~~',
 '우도 도착!!!!',
 '바람이 아주 그냥 휘몰아칩니다.',
 '제주도는 바람만 안불면 정말 좋은 여행지가 아닐까 라는 생각이 수천번은 들었습니다 ^^',
 '우선 우도여행을 위한 전기바이크를 대여합니다.',
 '어떻게 조작을 하는지 아주 주위 깊게 듣고 있죠 ㅋㅋ',
 '바람 때문에 개방되어 있지 않는 바이크도 있으나 부인님과 앞뒤로 타야하고 개방감도',
 '부족해서 반오픈형 바이크로 선택했습니다.',
 '그럼 달려 봅니다.',
 '운전면허증은 있으나 운전은 못하는 부인님은 신나서 바이크를 몰아 봅니다.',
 '섰더 멈췄다를 수없이 반복하여 허리 나가는 줄 알았지만',
 '즐거웠습니다 ^^',
 '바이크의 장점은 가는 길에 느낌 오는 곳에 내려서 사진찍고 즐기고 또 출발하고 자유롭네요',
 '#산호해수욕장 가는길에 그냥 내려서 사진 한방!!',
 '#산호해수욕장',
 '산호해수욕장 도착을 했는데',
 '하늘은 너무 맑고',
 '바다는 너무 투명하고',
 '백사장의 모래는 하얗고',
 '아주 아름다웠습니다.',
 '눈호강 제대로 했습니다.',
 '가만히 앉아서 바라보는 것만으로도 힐링이 되고',
 '평생에 받은 스트레스가 풀리는 감성스팟입니다.',
 '심지어 헬멧도 아주 예뻐 보이네요.',
 '조금 

In [255]:
for i in range(100):
    print(i, text['description'][i])

0 생각 덜어내기 여행이었기 때문에 차량 대여 보다는 가벼운 방법을 고심하다가, 여러 곳 중에서도... 봉끄랑 주소 : 제주시 우도면 우도해안길 144 제주 <b>우도 여행</b>은 자유 일정이기 때문에 섬에 들어오자마자... 
1 마지막 날은 오로지 우도  제주도 <b>우도 여행</b> 3일차 코스  성산항 - 물꼬 해녀의 집 - 블랑로쉐 - 비양도... 비쳤던 우도 바다 코시국의 여행템 언제 끝날까 코로나... 우도 해변에서 주워 온 조개 라이딩 하다가 안녕... 
2 이번 여행에서는 우도 전기차를 대여해서 편하게 놀다왔는데요. 사실 저는 성산에서 우도 가는... 제주 <b>우도 여행</b>에서는 제일 중요한 것이 배시간인데요. 항우목동항과 천진항 두 군데의 항구가 있는데, 출발은... 
3 <b>우도 여행</b>, 우도 버스투어 2021.2월 이번에 갔던 <b>우도 여행</b>에서 가볼만한곳을 정리해봅니다 - 요즘에는... 아무래도 금액적인 부분이 많이 세이브 되니까 이것도 무시 못하죠 ㅎㅎ 그럼 우도 가볼만한곳, 여행코... 
4 성산일출봉, <b>우도</b> 구경하고, 김녕, 함덕해수욕장 근처숙소 숙박, 비자림, 사려니숲, 공항도착, <b>여행</b> 끝. 비가 온다는 날씨예보가 무색하게 맑았다. 지역에 따라 흐렸다, 비가 잠시 내렸다 맑았다, 안개가 짙게... 
5 제주 <b>우도 여행</b>에서 가장 중요한 우도 배시간인데요. 천진항과 하우목동항 두 개의 항구가 있는데 30분 간격으로 출발하고 성산 우도 간에 교차 시간표에요. 매표 후 안내표지를 따라 밖으로 나왔는데요.... 
6 댕댕이와 함께한 제주도여행 4일차! 우도 한라산 여파로 일정을 타이트하게 잡지 않았고 점심쯤 일어나 여유있게 움직이기로 했는데 다음날 계획이 하필 <b>우도여행</b>이였어서 유모차를 들고가기엔 안에서... 
7 우도 배시간, 성산항에서 우도까지 배표요금 다 알아왔어요^^ 제주<b>우도여행</b>하시는 분들께 도움됐음 하네요! 우도가 요즘 정말 핫하더라구요!! 이제 제주

In [82]:
from konlpy.tag import Hannanum
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [155]:
text_mining = pd.DataFrame({
    'title':['title'],
    'desc':['desc']
})
text_mining

title  desc
0  title  desc

In [156]:
for name in text_main['title'].unique():
    desc = text_main[text_main['title']==name]['link']
    desc = desc.reset_index(drop = True)
    for i in range(1, len(desc)):
        desc[0] = str(desc[0])+str(desc[i])
    # text_data = re.sub('[0-9]+', '', text_data)
    desc[0] = re.sub('[A-Za-z]+', '', desc[0])
    desc[0] = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘’|\(\)\[\]\<\>`\'…》]', '', desc[0])
    con = [name, desc[0]]
    text_mining.loc[len(text_mining)] = con

In [157]:
text_mining.drop(0, inplace=True)
text_mining.reset_index(drop=True, inplace=True)
text_mining[text_mining['title']=='우도']['desc']

0    제주여행 5 0317 감성 가득한 섬속의 섬 우도    2021 6 11 1120 ...
Name: desc, dtype: object

In [214]:
text_mining['desc'][0]

'제주여행 5 0317 감성 가득한 섬속의 섬 우도    2021 6 11 1120 본문 기타 기능  이번 제주여행의 핵심이라면 핵심이라고 할 수 있는 우도여행입니다 섬속의 섬 사진과 영상으로만 봤던 에메랄드 빛 바다를 보러 갑니다 성산포종합여객터미널 에서 우도천진항으로 이동을 합니다 성산포종합여객터미널 우도천진항으로 우도가 보이네요 날씨가 정말 좋았습니다 구름이 드리운 날씨지만 우도 여행중에는 햇살이 따스하게 내렸습니다 거리는 멀지 않아 20분 정도면 도착을 하는듯 합니다 설레옵니다 우도 도착 바람이 아주 그냥 휘몰아칩니다 제주도는 바람만 안불면 정말 좋은 여행지가 아닐까 라는 생각이 수천번은 들었습니다  우선 우도여행을 위한 전기바이크를 대여합니다 어떻게 조작을 하는지 아주 주위 깊게 듣고 있죠 ㅋㅋ 바람 때문에 개방되어 있지 않는 바이크도 있으나 부인님과 앞뒤로 타야하고 개방감도 부족해서 반오픈형 바이크로 선택했습니다 그럼 달려 봅니다 운전면허증은 있으나 운전은 못하는 부인님은 신나서 바이크를 몰아 봅니다 섰더 멈췄다를 수없이 반복하여 허리 나가는 줄 알았지만 즐거웠습니다  바이크의 장점은 가는 길에 느낌 오는 곳에 내려서 사진찍고 즐기고 또 출발하고 자유롭네요 산호해수욕장 가는길에 그냥 내려서 사진 한방 산호해수욕장 산호해수욕장 도착을 했는데 하늘은 너무 맑고 바다는 너무 투명하고 백사장의 모래는 하얗고 아주 아름다웠습니다 눈호강 제대로 했습니다 가만히 앉아서 바라보는 것만으로도 힐링이 되고 평생에 받은 스트레스가 풀리는 감성스팟입니다 심지어 헬멧도 아주 예뻐 보이네요 조금 밝은 옷을 입고 갈걸 그랬습니다 전 아주 시커먼놈처럼 나왔네요 거진 가오나시 급이네요 가오나시 그래도 배경은 아주 그뤠잇입니다 아주 한적하고 평화롭고 아름다운 산호해수욕장입니다 다시 우도 투어를 하기 위해서 부인님은 운전 준비중입니다 고고 점심을 먹으로 하하호호 로 이동합니다 하하호호 이동하는 길도 너무 예쁘네요 가는길에 하우목동항 이 있어 잠깐 사진을 찍어 봅니다 하우목동항 하우목

In [222]:
okt = Okt()
def noun_count(row):
    noun = okt.nouns(row['desc'])
    count = Counter(noun)
    noun_list = count.most_common(50)
    return noun_list

In [223]:
text_noun = text_mining.copy()

title                                               desc
0                 우도  제주여행 5 0317 감성 가득한 섬속의 섬 우도    2021 6 11 1120 ...
1              성산일출봉  제주 광치기해변과 성산일출봉 동쪽 코스 여행 허니  허니 2021 6 12 600 ...
2              사려니숲길  제주 동쪽 가볼만한곳 당일치기 여행 사려니숲길 등 시후마미  시후마미 2021 4 ...
3              카멜리아힐  제주도 6월여행 제주 수국 축제 카멜리아힐 임비  임비 2021 5 29 2335 ...
4             협재해수욕장  유별나던 협재해수욕장 맛집 모음 김김  김김 2021 6 11 700 본문 기타 기...
...              ...                                                ...
1068  황우럭만화천국사회적협동조합  _201222081556298098142213991937490072211925451...
1069             효명사  제주도 숨은명소 제주 효명사 천국의 문 날아라 마녀씨  날아라 마녀씨 2021 5 ...
1070            후포해변  아들 둘과 즉흥으로 삼척울진 여행 울진편성류굴후포해변등기산 징여  징여 2020 7...
1071     휘닉스 르쏠레이테라피  서귀포스파 르쏠레이테라피   휘닉스아일랜드 제주도조아  제주도조아 2020 10 1...
1072              휴림  남양주 휴림캠핑장 붉은목련이 너무 이쁜 감성캠핑장 셋둘맘  셋둘맘 2021 4 19...

[1073 rows x 2 columns]

In [225]:
text_noun['desc'] = text_noun.apply(noun_count, axis = 1)
text_noun

title                                               desc
0                 우도  [(우도, 490), (여행, 162), (곳, 120), (수, 114), (항,...
1              성산일출봉  [(성산일출봉, 179), (제주, 172), (여행, 137), (곳, 124),...
2              사려니숲길  [(숲길, 229), (사려, 204), (제주, 156), (곳, 138), (수...
3              카멜리아힐  [(카멜리아, 236), (힐, 231), (수국, 161), (제주, 158), ...
4             협재해수욕장  [(제주, 169), (협재해수욕장, 141), (수, 118), (곳, 113),...
...              ...                                                ...
1068  황우럭만화천국사회적협동조합  [(만화, 32), (황우럭, 29), (사회, 26), (공간, 17), (것, ...
1069             효명사  [(효, 209), (명사, 183), (곳, 177), (제주, 164), (문,...
1070            후포해변  [(후포, 162), (해변, 148), (울진, 126), (곳, 89), (바다...
1071     휘닉스 르쏠레이테라피  [(스파, 237), (제주, 199), (르쏠, 194), (관리, 159), (...
1072              휴림  [(캠핑장, 167), (림, 157), (로봇, 144), (캠핑, 102), (...

[1073 rows x 2 columns]

In [229]:
text_noun['desc'][55]

[('보롬왓', 219),
 ('곳', 217),
 ('꽃', 183),
 ('사진', 162),
 ('수', 148),
 ('여행', 139),
 ('제주도', 139),
 ('제주', 129),
 ('튤립', 94),
 ('메밀', 92),
 ('유채꽃', 80),
 ('카페', 68),
 ('것', 66),
 ('밭', 62),
 ('정말', 56),
 ('볼', 53),
 ('깡통', 49),
 ('열차', 47),
 ('입장료', 46),
 ('사람', 45),
 ('바람', 41),
 ('시간', 41),
 ('수국', 40),
 ('코스', 39),
 ('풍경', 38),
 ('포토', 37),
 ('삼색', 36),
 ('분', 36),
 ('식물', 35),
 ('더', 34),
 ('버드나무', 34),
 ('날', 34),
 ('공간', 34),
 ('존', 34),
 ('보고', 33),
 ('꼭', 33),
 ('때', 33),
 ('보라', 33),
 ('저', 32),
 ('실내', 32),
 ('구경', 32),
 ('이', 31),
 ('안', 31),
 ('생각', 30),
 ('꽃밭', 30),
 ('오름', 30),
 ('우리', 29),
 ('길', 28),
 ('처음', 26),
 ('정원', 25)]

In [158]:
text_main['link']

0        [[제주여행 day5 03.17] 감성 가득한 섬속의 섬 우도, JK, , JK, ...
1        [투어로 떠나보는 제주 우도 여행, 동팔이, , 동팔이, 2021. 4. 1. 22...
2        [제주도 2박3일 렌트카 없이 여행 우도 코스, beige, , beige, 202...
3        [우도 전기차 대여해서 편하게 여행하기, 어신, , 어신, 2020. 12. 1. ...
4        [우도 버스투어 : 우도 가볼만한곳, 여행코스, 송송, , 송송, 2021. 4. ...
                               ...                        
22555    [[오토캠핑] 남양주 휴림 캠핑장(2019. 11. 2~3), 아직푸름, , 아직푸...
22556    [남양주 휴림캠핑장 서울 근교 깔끔한 캠핑장, 여밤, , 여밤, 2020. 12. ...
22557    [🏷 캠핑기록 : 남양주시 휴림캠핑장 서울 근교 캠핑장 추천 - 1편🏕, 한뿌뿌, ...
22558    [휴림로봇 주가 - 이낙연, 하왕, , 하왕, 2021. 1. 20. 2:30, 본...
22559    [5월 4일 특이종목 정리 - 휴림로봇, 에이티넘인베스트먼트, 코콤, 아미노로직스,...
Name: link, Length: 22560, dtype: object

In [152]:
def countCV(row):
    temp_cv = CountVectorizer(min_df=0.5)#ngram_range = (1))
    temp_cv.fit([row['desc']])
    sdict= sorted(temp_cv.vocabulary_.items())#, reverse=True)
    return sdict

In [153]:
text_mining['desc'] = text_mining.apply(countCV, axis=1)
text_mining

AttributeError: 'list' object has no attribute 'lower'

In [201]:
text_train = text_main.copy()
text_train

title                                               link
0        우도  [[제주여행 day5 03.17] 감성 가득한 섬속의 섬 우도, JK, , JK, ...
1        우도  [투어로 떠나보는 제주 우도 여행, 동팔이, , 동팔이, 2021. 4. 1. 22...
2        우도  [제주도 2박3일 렌트카 없이 여행 우도 코스, beige, , beige, 202...
3        우도  [우도 전기차 대여해서 편하게 여행하기, 어신, , 어신, 2020. 12. 1. ...
4        우도  [우도 버스투어 : 우도 가볼만한곳, 여행코스, 송송, , 송송, 2021. 4. ...
...     ...                                                ...
22555    휴림  [[오토캠핑] 남양주 휴림 캠핑장(2019. 11. 2~3), 아직푸름, , 아직푸...
22556    휴림  [남양주 휴림캠핑장 서울 근교 깔끔한 캠핑장, 여밤, , 여밤, 2020. 12. ...
22557    휴림  [🏷 캠핑기록 : 남양주시 휴림캠핑장 서울 근교 캠핑장 추천 - 1편🏕, 한뿌뿌, ...
22558    휴림  [휴림로봇 주가 - 이낙연, 하왕, , 하왕, 2021. 1. 20. 2:30, 본...
22559    휴림  [5월 4일 특이종목 정리 - 휴림로봇, 에이티넘인베스트먼트, 코콤, 아미노로직스,...

[22560 rows x 2 columns]

In [200]:
def dim(row):
    result = np.array(row['link'])
    result = pd.Series(result)
    return result

In [202]:
text_train['link'] = text_train.apply(dim, axis=1)
text_train

title                                               link
0        우도                  [제주여행 day5 03.17] 감성 가득한 섬속의 섬 우도
1        우도                                  투어로 떠나보는 제주 우도 여행
2        우도                           제주도 2박3일 렌트카 없이 여행 우도 코스
3        우도                               우도 전기차 대여해서 편하게 여행하기
4        우도                           우도 버스투어 : 우도 가볼만한곳, 여행코스
...     ...                                                ...
22555    휴림                   [오토캠핑] 남양주 휴림 캠핑장(2019. 11. 2~3)
22556    휴림                            남양주 휴림캠핑장 서울 근교 깔끔한 캠핑장
22557    휴림             🏷 캠핑기록 : 남양주시 휴림캠핑장 서울 근교 캠핑장 추천 - 1편🏕
22558    휴림                                      휴림로봇 주가 - 이낙연
22559    휴림  5월 4일 특이종목 정리 - 휴림로봇, 에이티넘인베스트먼트, 코콤, 아미노로직스, ...

[22560 rows x 2 columns]

In [204]:
X = text_train['link']
y = text_train['title']
X.shape, y.shape

((22560,), (22560,))

In [209]:
X

0                        [제주여행 day5 03.17] 감성 가득한 섬속의 섬 우도
1                                        투어로 떠나보는 제주 우도 여행
2                                 제주도 2박3일 렌트카 없이 여행 우도 코스
3                                     우도 전기차 대여해서 편하게 여행하기
4                                 우도 버스투어 : 우도 가볼만한곳, 여행코스
                               ...                        
22555                     [오토캠핑] 남양주 휴림 캠핑장(2019. 11. 2~3)
22556                              남양주 휴림캠핑장 서울 근교 깔끔한 캠핑장
22557               🏷 캠핑기록 : 남양주시 휴림캠핑장 서울 근교 캠핑장 추천 - 1편🏕
22558                                        휴림로봇 주가 - 이낙연
22559    5월 4일 특이종목 정리 - 휴림로봇, 에이티넘인베스트먼트, 코콤, 아미노로직스, ...
Name: link, Length: 22560, dtype: object

In [205]:
text_model = make_pipeline(CountVectorizer(),
                            LogisticRegression())
text_model.fit(X,y)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [236]:
cafe = pd.read_csv('./cafe_blog.csv',encoding='utf-8')
cafe

title                                               link
0        봄날카페  ['풍경이 예뻤던 제주 봄날카페', 'CoCo SIRI', '', 'CoCo SIR...
1        봄날카페  ['21’제주 DAY5 :: 애월 한담해안산책로, 카페 봄날, 김만복김밥 본점', ...
2        봄날카페  ['분위기 좋은 제주 봄날카페 &광해', '바실랑', '', '바실랑', '2020...
3        봄날카페  ['매력 넘치는 제주 봄날카페', '짱가', '', '짱가', '2020. 7. 1...
4        봄날카페  ['제주도 한담해안산책로 카약 & 일몰, 봄날카페', 'x까미마루쭈니맘x', '',...
...       ...                                                ...
4450  홍요깃거리카페                                                 []
4451  홍요깃거리카페                                                 []
4452  홍요깃거리카페                                                 []
4453  홍요깃거리카페  ['제주 이색 카페 - 홍요깃거리 세화점', '러블리쿄쿄', '', '러블리쿄쿄',...
4454  홍요깃거리카페                                                 []

[4455 rows x 2 columns]

In [237]:
cafe[cafe['title']=='홍요깃거리카페']

title                                               link
4444  홍요깃거리카페  ['제주 홍요깃거리카페 : 보말칼국수 (배말칼국수), 성게칼국수 먹으러 다녀왔어요 ...
4445  홍요깃거리카페                                                 []
4446  홍요깃거리카페  ['[제주도] 애월 얼큰한 보말칼국수, 홍요깃거리카페 바다전망 굿!!', '소시미'...
4447  홍요깃거리카페                                                 []
4448  홍요깃거리카페                                                 []
4449  홍요깃거리카페                                                 []
4450  홍요깃거리카페                                                 []
4451  홍요깃거리카페                                                 []
4452  홍요깃거리카페                                                 []
4453  홍요깃거리카페  ['제주 이색 카페 - 홍요깃거리 세화점', '러블리쿄쿄', '', '러블리쿄쿄',...
4454  홍요깃거리카페                                                 []

In [238]:
cafe_text = pd.DataFrame({
    'title':['title'],
    'desc':['desc']
})
cafe_text

title  desc
0  title  desc

In [239]:
for name in cafe['title'].unique():
    desc = cafe[cafe['title']==name]['link']
    desc = desc.reset_index(drop = True)
    for i in range(1, len(desc)):
        desc[0] = str(desc[0])+str(desc[i])
    # text_data = re.sub('[0-9]+', '', text_data)
    desc[0] = re.sub('[A-Za-z]+', '', desc[0])
    desc[0] = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘’|\(\)\[\]\<\>`\'…》]', '', desc[0])
    con = [name, desc[0]]
    cafe_text.loc[len(cafe_text)] = con

In [240]:
cafe_text.drop(0, inplace=True)
cafe_text.reset_index(drop=True, inplace=True)
cafe_text[cafe_text['title']=='봄날카페']['desc']

0    풍경이 예뻤던 제주 봄날카페      2020 8 13 1959 본문 기타 기능  ...
Name: desc, dtype: object

In [241]:
cafe_text['desc'][0]

'풍경이 예뻤던 제주 봄날카페      2020 8 13 1959 본문 기타 기능  \\200 \\200 비가 갠 후 날씨가 너무 좋아서 친구와 함께 제주 봄날카페에 다녀왔어요 워낙 소문난 곳이라 기대가 되었는데 아주 즐거운 시간을 보내고 왔답니다 \\200 제주 봄날카페  부두식당 \\200 \\200 제주 봄날은 카페거리에 위치해 있는데 이전에 드라마 배경으로 쓰이며 굉장한 유명세를 얻게 된 곳이에요 그만큼 예쁘게 꾸며져 있기도 하고 오션뷰가 보이기도 해서 아직까지 발길이 많이 닿는 곳이지요 \\200 \\200 선 주문 후 입장으로 음료를 먼저 주문해야만 입장이 가능한데요 그만큼 잘 관리되니 더욱 편히 온전한 휴식을 취할 수 있는 곳이었어요  이곳이 계산대인데 뭔가 매표소 같은 느낌이 드는 공간이죠 저희는 우선 음료만 주문하고 입장 후 디저트를 추가 주문하기로 했어요 \\200 \\200 그리고 안으로 들어가보니 가장 먼저 눈에 띈 것은 이 귀여운 웰시코기들인데 여기 마스코트인 것 같더라고요 팔자 좋게 누워 잠을 자는데 어찌나 귀여운지 몰라요 \\200 \\200 제주 봄날카페는 마치 작은 마을에 들어온 듯한 느낌이 들던 곳이었어요 양 옆으로 건물이 자리하고 가운데엔 동네 작은 골목길 같이 길이 나 있었어요 \\200 \\200 형형 색색의 알전구가 위쪽에 달려있어 분위기를 더하고 곳곳에 녹색 식물들을 배치해 싱그러움이 가득했어요 \\200 내부로 들어가보니 통 유리로 시원스레 뚫려 있었고 밝은 햇살이 스며 들어오더라고요 사람들이 아주 많은 편은 아니었어서 우리만의 조용한 시간을 보내기 좋은 공간이었어요 \\200 \\200 창밖을 바라보니 이렇게 푸른 바다가 쫙 펼쳐져 있는데 오션뷰가 대단하더군요 일렁이는 파도와 저 멀리 섬들까지 한 눈에 보이니 하염없이 밖만 쳐다보았네요 \\200 실내에서도 추가로 주문을 할 수 있었는데 커피 외에도 아이스티와 과일쥬스 에이드 등 카페인을 드시지 않는 분들을 위한 음료들도 많았어요 \\200 \\200 제주 봄날카페는

In [243]:
cafe_text['desc'] = cafe_text.apply(noun_count, axis = 1)
cafe_text

title                                               desc
0              봄날카페  [(카페, 179), (봄날, 126), (제주, 94), (곳, 85), (수, ...
1         몽상드애월(몽상)  [(제주, 121), (카페, 107), (수, 59), (몽상, 57), (애월,...
2    서연의집(영화 건축학개론)  [(카페, 145), (집, 98), (서연, 95), (영화, 86), (건축학개...
3              리치망고  [(망고, 139), (리치, 82), (제주, 75), (제주도, 60), (곳,...
4             카페공작소  [(카페, 161), (화, 72), (소, 67), (해변, 57), (공작, 5...
..              ...                                                ...
400           함덕카페h  [(카페, 77), (함덕, 73), (체험, 28), (제주도, 25), (청귤,...
401              해뜰  [(해뜰, 68), (체험, 46), (수, 43), (목장, 35), (아이, 2...
402          허니문하우스  [(제주, 81), (카페, 77), (곳, 69), (허니문, 65), (하우스,...
403            호도제과  [(빵, 160), (호도, 99), (제과, 98), (빵집, 64), (서귀포,...
404         홍요깃거리카페  [(칼국수, 26), (홍, 21), (깃, 21), (거리, 21), (카페, 2...

[405 rows x 2 columns]

In [258]:
for i in cafe_text['desc']:
    print(i)

[('카페', 179), ('봄날', 126), ('제주', 94), ('곳', 85), ('수', 83), ('것', 51), ('주문', 41), ('안', 41), ('애월', 39), ('산책로', 36), ('바다', 35), ('담', 34), ('사진', 32), ('이', 30), ('더', 30), ('시간', 29), ('해변', 29), ('거리', 28), ('때', 26), ('뷰', 25), ('방문', 24), ('담해', 22), ('사람', 21), ('분위기', 20), ('바로', 20), ('맛', 20), ('제주도', 20), ('느낌', 19), ('하나', 19), ('밥', 19), ('여기', 18), ('커피', 18), ('다음', 17), ('식사', 17), ('여행', 17), ('음료', 16), ('자리', 16), ('기분', 16), ('날', 16), ('갈치', 16), ('오션', 15), ('주차장', 15), ('입장', 14), ('먼저', 14), ('포토', 14), ('차', 14), ('후', 13), ('저희', 13), ('듯', 13), ('존', 13)]
[('제주', 121), ('카페', 107), ('수', 59), ('몽상', 57), ('애월', 54), ('곳', 53), ('드애월', 52), ('여기', 52), ('제주도', 47), ('몽상드애월', 41), ('해변', 41), ('바다', 41), ('사람', 39), ('안', 38), ('사진', 36), ('주문', 35), ('담', 31), ('자리', 31), ('것', 31), ('야외', 29), ('이', 29), ('음료', 29), ('더', 28), ('바로', 28), ('산책로', 25), ('여행', 23), ('뷰', 22), ('테라스', 22), ('앞', 20), ('생각', 20), ('날씨', 20), ('날', 20), ('테이블', 19), ('거리', 19), 

In [263]:
cafe_text['desc'][0][0][0]=='카페'

True

In [250]:
cafe = pd.read_csv('./cafe.csv', encoding = 'euc-kr')
cafe

음식점                          상세주소   Latitude   Longitude  \
0              봄날카페      제주특별자치도 제주시 애월읍 애월로1길 25  33.462459  126.309591   
1         몽상드애월(몽상)    제주특별자치도 제주시 애월읍 애월북서길 56-1  33.462809  126.309678   
2    서연의집(영화 건축학개론)     제주특별자치도 서귀포시 남원읍 위미해안로 86  33.269437  126.655663   
3              리치망고     제주특별자치도 제주시 애월읍 애월해안로 272  33.478223  126.366563   
4             카페공작소   제주특별자치도 제주시 구좌읍 해맞이해안로 1446  33.524648  126.859939   
..              ...                           ...        ...         ...   
402           함덕카페h    제주특별자치도 제주시 조천읍 함대로3길 24-9  33.522570  126.662783   
403              해뜰    제주특별자치도 서귀포시 성산읍 일출로 270-8  33.461847  126.934227   
404          허니문하우스      제주특별자치도 서귀포시 칠십리로 228-13  33.245288  126.578749   
405            호도제과  제주특별자치도 서귀포시 이어도로 923-1 호도제과  33.237883  126.513256   
406         홍요깃거리카페     제주특별자치도 제주시 애월읍 애월해안로 352  33.478223  126.366563   

               주소                       해시태그  \
0        제주시 > 애월                #카페 #커피 #음료   
1        제주시 > 애월                #카페 #커피 #음료   
2       서귀포시 > 남원             #카페 #커피 #아메리카노   
3        제주시 > 애월  #스페셜망고쉐이크 #망고라떼 #수제망고코코넛잽   
4        제주시 > 구좌            #카페 #수제케이크 #에이드   
..            ...                        ...   
402      제주시 > 조천      #수제 생과일주스 #마카롱 # 다쿠아즈   
403     서귀포시 > 성산            #브런치 #백년초음료 #카페   
404  서귀포시 > 서귀포시내               #부모 #커플 #사계절   
405  서귀포시 > 서귀포시내                 #빵집 # 베이커리   
406      제주시 > 애월         #카페 #보말칼국수 #한라봉 생면   

                                                  상세정보  \
0                            드라마 '맨도롱 또똣'의 주인공이 일하던 카페   
1             제주 애월 한담에 위치하여 멋진 바다경관을 감상할 수 있는 해안로의 카페   
2                영화 '건축학개론' 속 서연의 집을 리모델링하여 오픈한 카페&갤러리   
3                       제주 애월에 런칭하여 제주 전역으로 확장한 망고음료카페   
4             아름다운 세화 바다를 바라보면서 커피 한 잔의 여유를 즐길 수 있는 카페   
..                                                 ...   
402                         함덕에 위치한 감귤과수원내에 있는 조용한 카페.   
403                                   성산일출봉 풍경을 마시는 카페   
404          유럽풍 건축물과 파노라마로 펼쳐지는 바다 경관을 보며 즐기는 베이커리 카페   
405  서귀포 법환동에 위치한 호도제과\n유기농 밀과 좋은 재료를 고집하고 유화제와 개량제...   
406                                 카페에서 맛보는 고소한 보말칼국수   

                                                   이미지  
0                 /image/weather/weather_g_cloud_l.png  
1                 /image/weather/weather_g_cloud_l.png  
2    https://api.cdn.visitjeju.net/photomng/thumbna...  
3    https://api.cdn.visitjeju.net/photomng/thumbna...  
4    https://api.cdn.visitjeju.net/photomng/thumbna...  
..                                                 ...  
402               /image/weather/weather_g_cloud_l.png  
403               /image/weather/weather_g_cloud_l.png  
404  https://api.cdn.visitjeju.net/photomng/thumbna...  
405  https://api.cdn.visitjeju.net/photomng/thumbna...  
406  https://api.cdn.visitjeju.net/photomng/thumbna...  

[407 rows x 8 columns]

In [26]:
testtest = [1,2,3]

In [252]:
set(cafe['음식점']) - set(cafe_text['title'])

{'라붕제주 cafe', '파리바게트 제스코본점'}